> KRX 에서 종가, 거래량, PER, PBR, 투자자별 거래실적 가져오기

In [15]:
#!pip install pykrx
from pykrx import stock

In [3]:
# KOSPI 
stock_code = stock.get_market_ticker_list(market="KOSPI")
print(stock_code[:6], len(stock_code)) # 종목 개수

# 종목명 반환
stock_name = stock.get_market_ticker_name("005930")
stock_name

['095570', '006840', '027410', '282330', '138930', '001460'] 938


'삼성전자'

In [4]:
# 날짜
from datetime import date, timedelta

today = date.today().strftime('%Y%m%d')
yesterday = (date.today() - timedelta(1)).strftime('%Y%m%d')
beforeyday = (date.today() - timedelta(2)).strftime('%Y%m%d')

# 거래량

In [5]:
# 개별종목 시세 추이
df = stock.get_market_ohlcv_by_date(beforeyday,today, ticker="005930")
df

,시가,고가,저가,종가,거래량
날짜,,,,,
2021-09-30,74300,74800,73700,74100,19919361
2021-10-01,73900,74000,72900,73200,15704195


#### atr  : 전일종가,고가,저가 중 차이가 큰 값을 구하기

In [6]:
df_atr = df.rename(index={'날짜':'Date'}).drop(columns="시가", axis=1)
df_atr.columns= ['high', 'low','close','volume']
df_atr["atr"] = ""
df_atr

,high,low,close,volume,atr
날짜,,,,,
2021-09-30,74800,73700,74100,19919361,
2021-10-01,74000,72900,73200,15704195,


In [7]:
lst = []
atr = []
# 0->날짜, 1->종가, 2->고가, 3->저가
# 0->고가, 1->저가, 2->종가
for i in range(len(df_atr)-1):
    a = df_atr.iloc[i,0] - df_atr.iloc[i,1]  # 고가-저가
    b = df_atr.iloc[i,0] - df_atr.iloc[i+1,2]  # 고가-전날종가
    c = df_atr.iloc[i,1] - df_atr.iloc[i+1,2]  # 저가-전날종가
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))
df_atr.iloc[1,4] = atr
df_atr

,high,low,close,volume,atr
날짜,,,,,
2021-09-30,74800,73700,74100,19919361,
2021-10-01,74000,72900,73200,15704195,1600


# PER, PBR

In [8]:
# PER, PBR

df_per = stock.get_market_fundamental_by_date(yesterday, today, "005930")
df_per

,BPS,PER,PBR,EPS,DIV,DPS
날짜,,,,,,
2021-10-01,39406,19.06,1.86,3841,4.09,2994


In [9]:
df_per = df_per[['PER', 'PBR']]
df_per

,PER,PBR
날짜,,
2021-10-01,19.06,1.86


# 거래실적  
#### 업데이트 늦음

In [10]:
# 투자자별 거래실적

df_value = stock.get_market_trading_value_by_date(yesterday,today, "005930")
df_value

,기관합계,기타법인,개인,외국인합계,전체
날짜,,,,,
2021-10-01,-160317703400,1783387400,215103170300,-56568854300,0


In [11]:
df_value= df_value.drop(['전체'], axis = 1)
df_value

,기관합계,기타법인,개인,외국인합계
날짜,,,,
2021-10-01,-160317703400,1783387400,215103170300,-56568854300


# 컬럼 수정

In [12]:
import pandas as pd

df = pd.concat([df_atr, df_per, df_value],axis=1)
df

,high,low,close,volume,atr,PER,PBR,기관합계,기타법인,개인,외국인합계
날짜,,,,,,,,,,,
2021-09-30,74800,73700,74100,19919361,,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-01,74000,72900,73200,15704195,1600,19.06,1.86,-1.603177e+11,1.783387e+09,2.151032e+11,-5.656885e+10


In [13]:
df = df.reset_index()
df_new = df.rename(columns={'날짜': 'Date'})
df_new

,Date,high,low,close,volume,atr,PER,PBR,기관합계,기타법인,개인,외국인합계
0,2021-09-30,74800,73700,74100,19919361,,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-01,74000,72900,73200,15704195,1600,19.06,1.86,-1.603177e+11,1.783387e+09,2.151032e+11,-5.656885e+10


# 함수화

In [14]:
from pykrx import stock
import pandas as pd

class KRX:
    def getDate(self):
        self.end = date.today().strftime('%Y%m%d')  # 오늘
        self.start = (date.today() - timedelta(1)).strftime('%Y%m%d')  # 어제

    def getStock(self, code):
        self.ohlcv = stock.get_market_ohlcv_by_date(self.start, self.end, code).drop(['시가'], axis=1)
        self.funda = stock.get_market_fundamental_by_date(self.start, self.end, code)[['PER', 'PBR']]
        self.value = stock.get_market_trading_value_by_date(self.start, self.end, code).drop(['전체'], axis=1)

    def saving(self):
        self.df = pd.concat([self.ohlcv, self.funda, self.value], axis=1)
        self.df_krx = self.df.reset_index()
        self.df_krx.columns=['date', 'high','low','y','volume', 'per', 'pbr', 'institution', 'corp', 'retail', 'foreign']
        #self.df_krx = self.df_krx.set_index(drop=True)
        self.df_krx = self.df_krx.set_index('date')
        print(self.df_krx)
#         self.df_krx.to_sql('samsung', conn, if_exists='append') # 테이블명
#         conn.commit()  # db에 저장

if __name__ == "__main__":
    
    code = "005930"  # 종목코드
    krx = KRX()
    krx.getDate()
    krx.getStock(code)
    krx.saving()

             high    low      y    volume    per   pbr   institution  \
date                                                                   
2021-10-01  74000  72900  73200  15704195  19.06  1.86 -160317703400   

                  corp        retail      foreign  
date                                               
2021-10-01  1783387400  215103170300 -56568854300  
